In [14]:
import tensorflow as tf
print ("TensorFlow version: " + tf.__version__)
import matplotlib.pyplot as plt
import pandas as pd
import glob
import os

TensorFlow version: 2.17.0


Load Dataset
 Each File has 31 rows and 4 columns. The first 3 columns indicate heart rate, respiratory rate, and oxygen saturation of a baby; the last column indicates if the baby is in pain or not (0 means no pain, 1 means slight pain, 2 means severe pain, # means label is missing).


In [15]:
data_dir = "./Data/"
csv_files = glob.glob(data_dir + "/**/*.csv", recursive=True) # get all csv files in the data directory
combined_csv = pd.concat([pd.read_csv(f, header=None) for f in csv_files], ignore_index=True) # combine all csv files into one dataframe
combined_csv.drop(columns=[0], inplace=True) # drop the first column (index)
combined_csv = combined_csv[combined_csv.iloc[:, 3] != "#"] # remove rows with "#" in the 4th column
combined_csv # display the combined dataframe


,1,2,3,4
0,142,60,100,0
1,142,60,100,0
2,142,60,100,0
3,142,59,100,0
4,143,59,100,0
...,...,...,...,...
18657,169,19,98,1
18658,164,50,98,1
18659,164,74,98,1
18660,162,39,99,1


In [16]:
#  Shuffle dataset
from sklearn.utils import shuffle
combined_csv = shuffle(combined_csv)

In [ ]:
# Split Dataset
from sklearn.model_selection import train_test_split

train, valid = train_test_split(combined_csv, test_size=0.1) # split the dataset into 90% training and 10% validation (test)
